## 한국인의 영어 발화 데이터로 finetuning한  모델의 성능을 파악하기 위한 간단한 테스트 코드
총 3가지 예시 음성 파일이 있으며, 원하는 음성파일을 불러와서 직접 테스트 가능
## **기존 base 모델보다 성능 개선**

In [1]:
!pip install transformers
!pip install wave

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wave: filename=Wave-0.0.2-py3-none-any.whl size=1220 sha256=b15054ad53010e80933e9ddba81ed8a95f4c730b6ad23f96e95d1a4964012b1a
  Stored in directory: /root/.cache/pip/wheels/f8/24/4d/1b01c0e32da3eb3fd71bbbc6093fcc557ec3b2d9e532ecd65d
Successfully built wave


# Load data

In [2]:
!git clone https://github.com/pongjin/sample_wavs.git

Cloning into 'sample_wavs'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), 1.29 MiB | 7.34 MiB/s, done.


***
# 1. Predict 50sec native speech data


## change 8hz to 16hz

In [3]:
import wave
import torch
import scipy.signal as signal
import numpy as np

# WAV 파일 경로 지정
input_file = '/content/sample_wavs/male.wav'

# 입력 WAV 파일 열기
with wave.open(input_file, "rb") as wav_in:
    # 입력 WAV 파일의 샘플 속도 확인
    input_sample_rate = wav_in.getframerate()
    # 입력 WAV 파일의 오디오 데이터 읽기
    audio_data = wav_in.readframes(-1)
    audio_data = np.frombuffer(audio_data, dtype=np.int16)

# 입력 WAV 파일의 샘플 속도와 목표 샘플 속도 비율 계산
rate_ratio = 16000 / input_sample_rate

# 샘플 속도 변경
resampled_audio = signal.resample(audio_data, int(len(audio_data) * rate_ratio))

# 새로운 WAV 파일 생성
with wave.open(input_file, "wb") as wav_out:
    # 출력 WAV 파일 설정
    wav_out.setnchannels(1)  # 모노 오디오
    wav_out.setsampwidth(2)  # 16비트
    wav_out.setframerate(16000)  # 목표 샘플 속도

    # 오디오 데이터를 WAV 파일에 쓰기
    wav_out.writeframes(resampled_audio.astype(np.int16).tobytes())

## model input
1분을 한꺼번에 넣으면 메모리 부족으로 종료됨.  
따라서 10초씩 끊어서 진행( 추후 iterator로 메모리 사용 조절해야될듯..?)

In [4]:
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

# 사전 학습된 모델과 토크나이저 경로
model_name = "pongjin/en_with_korean_w2v_model_960h"
tokenizer_name = "pongjin/en_with_korean_w2v_model_960h"

# 모델과 토크나이저 로드
model = Wav2Vec2ForCTC.from_pretrained(model_name)
tokenizer = Wav2Vec2Tokenizer.from_pretrained(tokenizer_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:757: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# 앞서 16khz로 변경된 WAV 파일 로드
waveform, sample_rate = torchaudio.load(input_file)

# 잘라낼 시간 간격 설정 (초 단위)
interval = 10

# 잘라낸 오디오 데이터 저장할 리스트
sliced_audio_data = []

# 오디오 데이터를 interval 간격으로 자르기
start_time = 0
end_time = interval * sample_rate
while end_time <= waveform.size(1):
    sliced_audio_data.append(waveform[:, start_time:end_time])
    start_time += interval * sample_rate
    end_time += interval * sample_rate

## predict

In [6]:
total = []
# 잘라진 오디오 데이터를 모델의 입력값으로 사용
for sliced in sliced_audio_data:

    # 토큰화 및 인코딩
    input_values = tokenizer(sliced, return_tensors="pt").input_values
    input_values = torch.tensor(input_values[0])
    # 모델에 입력 텐서 전달하여 예측
    outputs = model(input_values)

    # CTC 디코딩을 통해 최종 예측 텍스트 얻기
    predicted_ids = outputs.logits.argmax(dim=-1)
    predicted_text = tokenizer.batch_decode(predicted_ids)[0]
    total.append(predicted_text)

total

<ipython-input-6-7a754bcb3072>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_values = torch.tensor(input_values[0])


["but what ive somebody decides to breake it  careful that you keep atequate coverage but look for places to save money maybe it's taking longer to get things s quared away than b",
 'ankers expect diring the wife or once company may win her taxated retirement and countboos elful  but inatequate new self saving rags are hurily t',
 'us don a to naked bones what a discussion can insue when the title of this type of song is in question theris no din or waxing or gassing nod nape weight may be personl',
 'asnon back while clay is lather hard place work on a flagt surface and smooth out the simplest kind of separate system uses it single suf contained uni',
 'to the old shop ats it still holds a good mechanic is usually a bad bus so figurs would go har in later year some make beautiful chairs cabinets cheks d']

In [8]:
# 예측
total[0]

"but what ive somebody decides to breake it  careful that you keep atequate coverage but look for places to save money maybe it's taking longer to get things s quared away than b"

In [7]:
# real sound
from IPython.display import Audio
Audio(filename='/content/sample_wavs/male.wav')

Output hidden; open in https://colab.research.google.com to view.

## 처음 10초만 transcript를 gpt로 변환한 결과
(change more naturally "문장")  

"But what if someone decides to break it? Make sure you maintain adequate coverage, but also explore opportunities to save money. Perhaps the process of getting everything sorted out is taking longer than expected."

## 아예 재창조함..?
1. 분명 인식 결과는 base보다 더 좋음. GPT한테 요청이 잘못 들어간것으로 파악

***
# 2. predict 10sec korean speech(already trained data)

In [9]:
# pcm to wav
import soundfile as sf
import librosa as lr

with open('/content/sample_wavs/SW201105ETRNGSF04623LWJ0073.pcm', 'rb') as tf:
    buf = tf.read()
    buf = buf+b'0' if len(buf)%2 else buf

pcm_data = np.frombuffer(buf, dtype='int16')
wav_data = lr.util.buf_to_float(x=pcm_data, n_bytes=2)
sf.write('/content/sample_wavs/SW201105ETRNGSF04623LWJ0073.wav', wav_data, 16000, format='WAV', endian='LITTLE', subtype='PCM_16')

이미 16hz라 변환 필요 없음

In [10]:
# 앞서 16khz로 변경된 WAV 파일 로드
waveform, sample_rate = torchaudio.load('/content/sample_wavs/SW201105ETRNGSF04623LWJ0073.wav')

# 잘라낼 시간 간격 설정 (초 단위)
interval = 10

# 잘라낸 오디오 데이터 저장할 리스트
sliced_audio_data = []

# 오디오 데이터를 interval초 간격으로 자르기
start_time = 0
end_time = interval * sample_rate
while end_time <= waveform.size(1):
    sliced_audio_data.append(waveform[:, start_time:end_time])
    start_time += interval * sample_rate
    end_time += interval * sample_rate

In [11]:
total = []
# 잘라진 오디오 데이터를 모델의 입력값으로 사용
for sliced in sliced_audio_data:

    # 토큰화 및 인코딩
    input_values = tokenizer(sliced, return_tensors="pt").input_values
    input_values = torch.tensor(input_values[0])
    # 모델에 입력 텐서 전달하여 예측 수행
    outputs = model(input_values)

    # CTC 디코딩을 통해 최종 예측 텍스트 얻기
    predicted_ids = outputs.logits.argmax(dim=-1)
    predicted_text = tokenizer.batch_decode(predicted_ids)[0]
    total.append(predicted_text)

<ipython-input-11-4b480cc60090>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_values = torch.tensor(input_values[0])


In [12]:
# 예측
total

['i waited an hour for my date but it was obvious that she stood me up']

In [13]:
# 음성파일
from IPython.display import Audio
Audio(filename='/content/sample_wavs/SW201105ETRNGSF04623LWJ0073.wav')

***
# 3. librispeech sample

In [14]:
!wget https://cdn-media.huggingface.co/speech_samples/sample1.flac

--2023-06-07 08:30:56--  https://cdn-media.huggingface.co/speech_samples/sample1.flac
Resolving cdn-media.huggingface.co (cdn-media.huggingface.co)... 143.204.146.8, 143.204.146.39, 143.204.146.101, ...
Connecting to cdn-media.huggingface.co (cdn-media.huggingface.co)|143.204.146.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 282378 (276K) [audio/flac]
Saving to: ‘sample1.flac’

sample1.flac        100%[===================>] 275.76K  --.-KB/s    in 0.03s   

2023-06-07 08:30:56 (10.3 MB/s) - ‘sample1.flac’ saved [282378/282378]



In [18]:
import soundfile as sf

REQUIRED_SAMPLE_RATE = 16000

def read_flac_file(file_path):
  with open(file_path, "rb") as f:
      audio, sample_rate = sf.read(f)
  return audio

audio =  read_flac_file('/content/sample1.flac')
audio

In [29]:
input_values = tokenizer(audio[0], return_tensors="pt").input_values
input_values = torch.tensor(input_values)

outputs = model(input_values)

# CTC 디코딩을 통해 최종 예측 텍스트 얻기
predicted_ids = outputs.logits.argmax(dim=-1)
predicted_text = tokenizer.batch_decode(predicted_ids)[0]
predicted_text

"hgoing along slushy country roads speaking to damp audiences in draugty school rooms day after day pr fortnight 'l have to put in an appearance some place of worship on sunday morning and he can come to us immediately afterwards"

In [15]:
from IPython.display import Audio
import random

Audio(filename='/content/sample1.flac')